# Train an IntegratedML model to Predict Hospital Readmission
## Using UCI Diabetes 130-US Hospitals Dataset

This Notebook demonstrates:
- Using the `intersystems-irispython` DB-API driver to connect to InterSystems IRIS
- Creating views to segment data into training and test sets with balanced sampling
- Defining and training an IntegratedML model to predict 30-day hospital readmissions
- Comparing the resulting model's predictions to data in the test set
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics

**Dataset:** UCI Machine Learning Repository - Diabetes 130-US Hospitals for Years 1999-2008

This dataset contains ~100,000 hospital admission records for diabetic patients, with a `readmitted` column indicating:
- `NO` - No readmission
- `>30` - Readmitted after 30 days
- `<30` - Readmitted within 30 days (our target for prediction)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1. Connect to InterSystems IRIS using DB-API Driver

In [ ]:
# InterSystems IRIS DB-API Driver connection
# Using the official intersystems-irispython package from PyPI
import iris
import pandas as pd

# Connection configuration
connection_string = "irisimlsvr:1972/USER"
username = "SUPERUSER"
password = "SYS"

# Establish connection
conn = iris.connect(connection_string, username, password)
curs = conn.cursor()
print("Connected to InterSystems IRIS successfully!")

### 2. Define table names for our workflow

In [ ]:
# Source data table (loaded from UCI Diabetes dataset)
dataTable = 'Diabetes.Readmission'

# Views we'll create for training and testing
TargetTable = 'Diabetes.ReadmissionTarget'
TrainTable = 'Diabetes.ReadmissionTraining'
TestTable = 'Diabetes.ReadmissionTesting'

### 3. Explore the source data

In [ ]:
# Preview the data
df = pd.read_sql("SELECT TOP 10 * FROM %s" % dataTable, conn)
display(df)
print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Show total record count
df_count = pd.read_sql("SELECT COUNT(*) as TotalRecords FROM %s" % dataTable, conn)
display(df_count)

In [ ]:
# Show distribution of readmission values
# Note: 'Count' is a reserved word in IRIS SQL, so we use 'RecCount' instead
df_dist = pd.read_sql("""
    SELECT readmitted, COUNT(*) as RecCount 
    FROM %s 
    GROUP BY readmitted
""" % dataTable, conn)
display(df_dist)
print("\nNote: '<30' means readmitted within 30 days (our positive class)")

### 4. Clean up any previous runs (if needed)

In [ ]:
# Uncomment these lines to clean up before re-running
# curs.execute("DROP VIEW %s" % TrainTable)
# curs.execute("DROP VIEW %s" % TestTable)
# curs.execute("DROP VIEW %s" % TargetTable)
# curs.execute("DROP MODEL ReadmitModel")

### 5. Create Target View with Binary Classification

We'll create a binary target variable:
- `1` = Readmitted within 30 days (`<30`)
- `0` = Not readmitted within 30 days (`NO` or `>30`)

We'll also select the most relevant features for prediction.

In [ ]:
# Create the target view with binary classification and selected features
# Note: Column names have underscores (e.g., time_in_hospital, not timeinhospital)
curs.execute("""
    CREATE VIEW %s AS 
    SELECT 
        ID,
        CASE WHEN LEFT(readmitted,1) = '<' THEN 1 ELSE 0 END AS ReadmitWithin30,
        age,
        gender,
        race,
        time_in_hospital,
        num_medications,
        number_emergency,
        number_inpatient,
        number_outpatient,
        number_diagnoses,
        num_lab_procedures,
        num_procedures,
        diabetesMed,
        insulin,
        metformin
    FROM %s
""" % (TargetTable, dataTable))

# Verify the view
df_target = pd.read_sql("SELECT TOP 5 * FROM %s" % TargetTable, conn)
display(df_target)

In [ ]:
# Check target variable distribution in our view
df_dist2 = pd.read_sql("""
    SELECT ReadmitWithin30, COUNT(*) as RecCount 
    FROM %s 
    GROUP BY ReadmitWithin30
""" % TargetTable, conn)
display(df_dist2)

total = df_dist2['RecCount'].sum()
positive = df_dist2[df_dist2['ReadmitWithin30'] == 1]['RecCount'].values[0]
print(f"\nPositive class rate: {positive/total*100:.2f}%")

### 6. Create Training and Testing Views

Split the data into 80% training and 20% testing using ID-based partitioning.

In [ ]:
# Get the max ID to determine split point
max_id_df = pd.read_sql("SELECT MAX(ID) as MaxID FROM %s" % TargetTable, conn)
max_id = max_id_df['MaxID'].iloc[0]
split_point = int(max_id * 0.8)

print(f"Max ID: {max_id}")
print(f"Split point (80% training): {split_point}")

In [ ]:
# Create Training view (80% of data)
curs.execute("""
    CREATE VIEW %s AS 
    SELECT * FROM %s 
    WHERE ID <= %d
""" % (TrainTable, TargetTable, split_point))

# Create Testing view (20% of data)
curs.execute("""
    CREATE VIEW %s AS 
    SELECT * FROM %s 
    WHERE ID > %d
""" % (TestTable, TargetTable, split_point))

# Verify splits
train_count = pd.read_sql("SELECT COUNT(*) as RecCount FROM %s" % TrainTable, conn)
test_count = pd.read_sql("SELECT COUNT(*) as RecCount FROM %s" % TestTable, conn)

print(f"Training records: {train_count['RecCount'].iloc[0]}")
print(f"Testing records: {test_count['RecCount'].iloc[0]}")

### 7. Create and Train the IntegratedML Model

In [ ]:
# Set the ML configuration to use AutoML
curs.execute("SET ML CONFIGURATION %AutoML")

In [ ]:
# Create the model - predicting ReadmitWithin30 (binary: 0 or 1)
curs.execute("""
    CREATE MODEL ReadmitModel 
    PREDICTING (ReadmitWithin30) 
    FROM %s
""" % TrainTable)

print("Model created successfully!")

In [ ]:
# Train the model using AutoML
# This will take a few minutes as AutoML tries different algorithms
print("Training model... this may take a few minutes...")
curs.execute("TRAIN MODEL ReadmitModel")
print("Training complete!")

In [ ]:
# View information about the trained model
df_models = pd.read_sql("""
    SELECT MODEL_NAME, TRAINED_MODEL_NAME, PROVIDER, MODEL_TYPE, MODEL_INFO 
    FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS
    WHERE MODEL_NAME = 'ReadmitModel'
""", conn)
display(df_models)

### 8. Compare Model Predictions to Actual Data

Now we test the model on data it has never seen (the test set).

In [ ]:
# Get predictions vs actual values on test set
df_predictions = pd.read_sql("""
    SELECT 
        PREDICT(ReadmitModel) AS PredictedReadmit,
        ReadmitWithin30 AS ActualReadmit
    FROM %s
""" % TestTable, conn)

display(df_predictions)

In [ ]:
# Create a confusion matrix
confusion = pd.crosstab(
    df_predictions['ActualReadmit'], 
    df_predictions['PredictedReadmit'],
    rownames=['Actual'],
    colnames=['Predicted']
)
print("Confusion Matrix:")
display(confusion)

In [ ]:
# Calculate accuracy metrics manually
correct = (df_predictions['PredictedReadmit'] == df_predictions['ActualReadmit']).sum()
total = len(df_predictions)
accuracy = correct / total * 100

print(f"\nManual Accuracy Calculation:")
print(f"Correct predictions: {correct}")
print(f"Total predictions: {total}")
print(f"Accuracy: {accuracy:.2f}%")

### 9. VALIDATE MODEL - IntegratedML's Built-in Metrics

IntegratedML provides comprehensive validation metrics automatically.

In [ ]:
# Run VALIDATE MODEL to generate accuracy metrics
curs.execute("VALIDATE MODEL ReadmitModel FROM %s" % TestTable)

# Get the most recent validation run for this model
df_metrics = pd.read_sql("""
    SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS 
    WHERE MODEL_NAME = 'ReadmitModel'
    ORDER BY VALIDATION_RUN_NAME DESC
""", conn)

# Get only the latest validation run and remove any duplicate metrics
latest_run = df_metrics['VALIDATION_RUN_NAME'].iloc[0]
df_latest = df_metrics[df_metrics['VALIDATION_RUN_NAME'] == latest_run].drop_duplicates(
    subset=['VALIDATION_RUN_NAME', 'METRIC_NAME'], keep='first'
)

# Pivot to display metrics in a readable format
df_pivot = df_latest.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df_pivot)

### 10. Query High-Risk Patients Using PROBABILITY()

Find patients with high probability of 30-day readmission.

In [ ]:
# Get patients with high readmission probability
df_highrisk = pd.read_sql("""
    SELECT 
        ID,
        PROBABILITY(ReadmitModel FOR '1') AS ReadmitProbability,
        PREDICT(ReadmitModel) AS PredictedReadmit,
        ReadmitWithin30 AS ActualReadmit,
        age,
        time_in_hospital,
        num_medications,
        number_emergency,
        number_inpatient
    FROM %s
    WHERE PROBABILITY(ReadmitModel FOR '1') >= 0.3
    ORDER BY PROBABILITY(ReadmitModel FOR '1') DESC
""" % TestTable, conn)

print(f"Found {len(df_highrisk)} high-risk patients (probability >= 30%)")
display(df_highrisk.head(20))

### 11. Analyze Misclassified Cases

Look at cases where the model was wrong - this helps understand model limitations.

In [ ]:
# Find false negatives - patients who were readmitted but model predicted they wouldn't be
df_false_neg = pd.read_sql("""
    SELECT 
        PROBABILITY(ReadmitModel FOR '1') AS ReadmitProbability,
        PREDICT(ReadmitModel) AS PredictedReadmit,
        ReadmitWithin30 AS ActualReadmit,
        age,
        gender,
        time_in_hospital,
        num_medications,
        number_emergency,
        number_inpatient
    FROM %s
    WHERE ReadmitWithin30 = 1 
      AND PREDICT(ReadmitModel) = 0
""" % TestTable, conn)

print(f"False Negatives (missed readmissions): {len(df_false_neg)}")
display(df_false_neg.head(15))

In [ ]:
# Find false positives - patients predicted to readmit but didn't
df_false_pos = pd.read_sql("""
    SELECT 
        PROBABILITY(ReadmitModel FOR '1') AS ReadmitProbability,
        PREDICT(ReadmitModel) AS PredictedReadmit,
        ReadmitWithin30 AS ActualReadmit,
        age,
        gender,
        time_in_hospital,
        num_medications,
        number_emergency,
        number_inpatient
    FROM %s
    WHERE ReadmitWithin30 = 0 
      AND PREDICT(ReadmitModel) = 1
""" % TestTable, conn)

print(f"False Positives (predicted readmit but didn't): {len(df_false_pos)}")
display(df_false_pos.head(15))

### Summary

This notebook demonstrated:

1. **Data Preparation**: Converting the UCI Diabetes dataset's readmission categories into a binary classification problem
2. **Train/Test Split**: Using SQL views to partition data for training and validation
3. **Model Training**: Using IntegratedML's AutoML to automatically select and train the best model
4. **Prediction**: Using `PREDICT()` to get model predictions on new data
5. **Validation**: Using `VALIDATE MODEL` to get comprehensive accuracy metrics
6. **Risk Scoring**: Using `PROBABILITY()` to identify high-risk patients for intervention

**Clinical Application**: In a real healthcare setting, this model could be used to:
- Identify patients at risk of early readmission before discharge
- Prioritize follow-up care and resources
- Trigger care coordination interventions for high-risk patients